In [1]:
import os
import h5py
import cortex
import copy
import numpy as np
import matplotlib.pyplot as plt

## Setup results path

In [2]:
base_path = "/Users/mosorio/Documents/CAJAL-NeuroAI/Project/cajal_llm_project/results/"
experiment_name = "shuffled_words/percentage0.25"
experiment_baseline_name = "original_code"
results_path = os.path.join(base_path, experiment_name)
results_path
results_baseline_path = os.path.join(base_path, experiment_baseline_name)
#results_baseline_path

In [3]:
cortex.database.default_filestore

'/Users/mosorio/miniforge3/envs/llms/share/pycortex/db'

## Setup fMRI data from individual

In [4]:
surfaces = dict(
# F = 'fMRI_story_F',
# G = 'fMRI_story_G',
# H = 'fMRI_story_H',
I = 'fMRI_story_I',
# J = 'fMRI_story_J',
# K = 'fMRI_story_K',
# L = 'fMRI_story_L',
# M = 'fMRI_story_M',
# N = 'fMRI_story_N'
)

transforms = dict(
# F = 'F_ars',
# G = 'G_ars',
# H = 'H_ars',
I = 'I_ars',
# J = 'J_ars',
# K = 'K_ars',
# L = 'L_ars',
# M = 'M_ars',
# N = 'N_ars'
)

new_transforms = dict(
# F = 'F_ars_auto2',
# G = 'G_ars_auto2',
# H = 'H_ars_auto2',
I = 'I_ars_auto2',
# J = 'J_ars_auto2',
# K = 'K_ars_auto2',
# L = 'L_ars_auto2',
# M = 'M_ars_auto2',
# N = 'N_ars_auto2'
)

sub = 'I'

In [5]:
# check whether these transforms work for the PLoS data
mask = cortex.db.get_mask(surfaces[sub], transforms[sub], 'thin')
print('num voxels in transform for I: {}'.format(np.sum(mask)))

num voxels in transform for I: 25281


In [6]:
mask = cortex.db.get_mask(surfaces[sub], new_transforms[sub], 'thin')
print('num voxels in new_transform for I: {}'.format(np.sum(mask)))

num voxels in new_transform for I: 25263


In [7]:
# Open in read mode
with h5py.File(os.path.join(results_path, "results_encoding_model.h5"), 'r') as f:
    # Read metadata
    model_name = f.attrs['model_name']
    print("Model name:", model_name)

    # List all groups
    print("Groups available:", list(f.keys()))

    # Choose your group
    representation_name = list(f.keys())[0]  # or set explicitly
    group = f[representation_name]

    # Read datasets
    predictions = group['predictions'][:]
    ground_truth = group['ground_truth'][:]
    correlations = group['correlations'][:]
    p_values = group['p_values'][:]
    coefficients = group['coefficients'][:]
    alphas = group['alphas'][:]

    print("Predictions shape:", predictions.shape)

Model name: Llama-3.2-1B-Full-Chapter
Groups available: ['layer -7']
Predictions shape: (300, 25263)


In [8]:
# Open in read mode
with h5py.File(os.path.join(results_baseline_path, "results_encoding_model.h5"), 'r') as f:
    # Read metadata
    model_name_baseline = f.attrs['model_name']
    print("Model name:", model_name_baseline)

    # List all groups
    print("Groups available:", list(f.keys()))

    # Choose your group
    representation_name_baseline = list(f.keys())[0]  # or set explicitly
    group = f[representation_name_baseline]

    # Read datasets
    predictions_baseline = group['predictions'][:]
    ground_truth_baseline = group['ground_truth'][:]
    correlations_baseline = group['correlations'][:]
    p_values_baseline = group['p_values'][:]
    coefficients_baseline = group['coefficients'][:]
    alphas_baseline = group['alphas'][:]

    print("Predictions shape:", predictions_baseline.shape)

Model name: Llama-3.2-1B-Full-Chapter
Groups available: ['layer -7']
Predictions shape: (300, 25263)


In [9]:
dif_corr = correlations_baseline - correlations

In [10]:
correlations_baseline.mean()

np.float32(0.12602845)

In [11]:
np.sum(correlations_baseline > 0.2)

np.int64(6243)

In [12]:
correlations.mean()

np.float32(0.092566386)

In [13]:
np.sum(correlations > 0.2)

np.int64(3572)

In [14]:
# significant_voxels = np.where(p_values < 0.05, voxel_values_to_plot, 0.)
significant_voxels_baseline = copy.deepcopy(correlations_baseline)
significant_voxels_baseline[p_values > 0.01] = 0
significant_voxels_baseline.shape

(25263,)

In [15]:
# significant_voxels = np.where(p_values < 0.05, voxel_values_to_plot, 0.)
significant_voxels = copy.deepcopy(correlations)
significant_voxels[p_values > 0.01] = 0
significant_voxels.shape

(25263,)

In [16]:
# significant_voxels = np.where(p_values < 0.05, voxel_values_to_plot, 0.)
significant_voxels_diff = copy.deepcopy(dif_corr)
significant_voxels_diff[p_values > 0.01] = 0
significant_voxels_diff[p_values_baseline > 0.01] = 0
significant_voxels_diff.shape

(25263,)

## Plot all values

In [17]:
# vol = cortex.Volume(correlations, surfaces[sub], new_transforms[sub], vmin=0, vmax=0.6, cmap='viridis')
# cortex.webshow(vol)

## Plot significant values

In [20]:
label_volume = np.zeros(significant_voxels.shape, dtype=np.int32)

p_value_thresh=0.01

only_current   = (p_values >= 0) & (p_values <= p_value_thresh) & (p_values_baseline >= p_value_thresh)
only_baseline  = (p_values_baseline >= 0) & (p_values_baseline <= p_value_thresh) & (p_values >= p_value_thresh)
intersection   = (p_values >= 0) & (p_values <= p_value_thresh) & (p_values_baseline >= 0) & (p_values_baseline <= p_value_thresh)

label_volume[only_current]   = 1
label_volume[only_baseline]  = 2
label_volume[intersection]   = 3

# Create cortex.Volume
vol = cortex.Volume(
    data=label_volume,
    subject=surfaces[sub],                     
    xfmname=new_transforms[sub],              
    vmin=0,
    vmax=3,
    cmap='viridis'
)

# Show in browser
cortex.webshow(vol)

Started server on port 17451
Stopping server


<JS: window.viewer>